In [2]:
# 导入必要的库
import os
import importlib
from datetime import datetime
import pandas as pd
from src.pipeline.hyper_heuristics.single import SingleHyperHeuristic
from src.util.util import search_file

# --------------------------
# 参数设置
# --------------------------
problem = "psp"  # 问题类型
heuristic_dir = "basic_heuristics"  # 启发式函数目录
test_data = "test_data"  # 测试数据目录
result_dir = "result"  # 结果保存目录

# --------------------------
# 获取所有.py启发式算法
# --------------------------
heuristic_pool_path = os.path.join("src", "problems", problem, "heuristics", heuristic_dir)
if not os.path.exists(heuristic_pool_path):
    raise FileNotFoundError(f"启发式目录不存在: {heuristic_pool_path}")

# 获取所有.py文件(排除__init__.py和.ipynb_checkpoints)
all_heuristics = [
    f.replace('.py', '') for f in os.listdir(heuristic_pool_path)
    if f.endswith('.py') and f != '__init__.py' and not f.startswith('.')
]

print(f"找到 {len(all_heuristics)} 个启发式算法: {all_heuristics}")

# --------------------------
# 获取所有测试数据
# --------------------------
test_data_dir = search_file("test_data", problem)
test_data_list = [
    f for f in os.listdir(test_data_dir)
    if f != ".ipynb_checkpoints" and not f.startswith('.')
]

print(f"找到 {len(test_data_list)} 个测试数据: {test_data_list}")

# --------------------------
# 导入环境类
# --------------------------
try:
    module = importlib.import_module(f"src.problems.{problem}.env")
    Env = getattr(module, "Env")
except Exception as e:
    raise ImportError(f"无法导入问题 {problem} 的环境类: {str(e)}")

# --------------------------
# 批量测试所有算法
# --------------------------
base_output_dir = os.path.join(os.getenv("AMLT_OUTPUT_DIR"), "..", "..", "output") if os.getenv("AMLT_OUTPUT_DIR") else "output"
datetime_str = datetime.now().strftime("%Y%m%d_%H%M%S")

# 存储结果的字典
results_dict = {
    'algorithm': []
}

# 为每个测试数据创建两列(目标函数值和计算时间)
for data_name in test_data_list:
    data_ref = data_name.split('.')[0]  # 去除扩展名
    results_dict[f'{data_ref}_目标函数'] = []
    results_dict[f'{data_ref}_计算时间'] = []

print("\n开始批量测试...")
print("=" * 80)

# 遍历所有启发式算法
for heuristic_name in all_heuristics:
    print(f"\n测试算法: {heuristic_name}")
    print("-" * 80)
    
    results_dict['algorithm'].append(heuristic_name)
    
    # 初始化单一启发式实例
    hyper_heuristic = SingleHyperHeuristic(heuristic=heuristic_name, problem=problem)
    
    # 在每个测试数据上测试
    for data_name in test_data_list:
        data_ref = data_name.split('.')[0]
        
        try:
            # 初始化环境
            env = Env(data_name=data_name)
            experiment_name = f"{heuristic_name}.{datetime_str}"
            output_dir = os.path.join(base_output_dir, problem, result_dir, env.data_ref_name, experiment_name)
            env.reset(output_dir)
            
            # 记录开始时间
            start_time = datetime.now()
            
            # 运行算法
            validation_result = hyper_heuristic.run(env)
            
            # 计算运行时间
            elapsed_time = (datetime.now() - start_time).total_seconds()
            
            # 保存结果
            if validation_result:
                env.dump_result()
                objective_value = env.key_value
                results_dict[f'{data_ref}_目标函数'].append(objective_value)
                results_dict[f'{data_ref}_计算时间'].append(f"{elapsed_time:.2f}s")
                print(f"  ✓ {data_name}: 目标={objective_value}, 时间={elapsed_time:.2f}s")
            else:
                results_dict[f'{data_ref}_目标函数'].append("无效解")
                results_dict[f'{data_ref}_计算时间'].append("-")
                print(f"  ✗ {data_name}: 无效解")
                
        except Exception as e:
            results_dict[f'{data_ref}_目标函数'].append("错误")
            results_dict[f'{data_ref}_计算时间'].append("-")
            print(f"  ✗ {data_name}: 错误 - {str(e)}")

# --------------------------
# 生成DataFrame和Excel
# --------------------------
df_results = pd.DataFrame(results_dict)

# 设置算法名称为索引
df_results.set_index('algorithm', inplace=True)

# 保存为Excel
excel_output_path = os.path.join(base_output_dir, problem, result_dir, f"batch_test_results_{datetime_str}.xlsx")
os.makedirs(os.path.dirname(excel_output_path), exist_ok=True)

df_results.to_excel(excel_output_path, engine='openpyxl')

print("\n" + "=" * 80)
print(f"测试完成! 结果已保存至: {excel_output_path}")
print("=" * 80)

# 显示结果预览
print("\n结果预览:")
print(df_results)

# 返回DataFrame供后续使用
df_results

找到 4 个启发式算法: ['cost_minimizing_greedy_cmg_b900', 'earliest_arrival_first_eaf_4fe1', 'largest_vessel_first_lvf_5e04', 'randomized_sequential_rs_4159']
找到 4 个测试数据: ['100_15_30_T24.txt', '100_15_30_T48.txt', '150_20_40_T24.txt', '150_20_40_T48.txt']

开始批量测试...

测试算法: cost_minimizing_greedy_cmg_b900
--------------------------------------------------------------------------------
  ✓ 100_15_30_T24.txt: 目标=69613.37834585927, 时间=0.40s
  ✓ 100_15_30_T48.txt: 目标=84264.42799270946, 时间=0.83s
  ✓ 150_20_40_T24.txt: 目标=105048.6807446939, 时间=1.22s
  ✓ 150_20_40_T48.txt: 目标=121209.46123499678, 时间=1.83s

测试算法: earliest_arrival_first_eaf_4fe1
--------------------------------------------------------------------------------
  ✓ 100_15_30_T24.txt: 目标=100147.94379169242, 时间=0.80s
  ✓ 100_15_30_T48.txt: 目标=120034.37065451941, 时间=0.70s
  ✓ 150_20_40_T24.txt: 目标=151701.68435417098, 时间=1.21s
  ✓ 150_20_40_T48.txt: 目标=187004.7834836999, 时间=3.89s

测试算法: largest_vessel_first_lvf_5e04
-----------------------------

,100_15_30_T24_目标函数,100_15_30_T24_计算时间,100_15_30_T48_目标函数,100_15_30_T48_计算时间,150_20_40_T24_目标函数,150_20_40_T24_计算时间,150_20_40_T48_目标函数,150_20_40_T48_计算时间
algorithm,,,,,,,,
cost_minimizing_greedy_cmg_b900,69613.378346,0.40s,84264.427993,0.83s,105048.680745,1.22s,121209.461235,1.83s
earliest_arrival_first_eaf_4fe1,100147.943792,0.80s,120034.370655,0.70s,151701.684354,1.21s,187004.783484,3.89s
largest_vessel_first_lvf_5e04,105915.667202,0.03s,121453.960881,0.03s,150457.144798,0.68s,184563.005071,1.20s
randomized_sequential_rs_4159,63798.123756,0.73s,81616.801942,1.12s,97898.908328,3.20s,121658.000518,2.88s
